In [21]:
%matplotlib inline
%config Completer.use_jedi = False

In [22]:
class settings:
    PINATA_CONFIG = {
        'api_key': "28cd3ecbe26a616a473c",
        'secret_api_key': "..."
    }

In [23]:
# pinata python sdk
from typing import Optional

import requests
from pydantic import BaseModel, validator
import datetime
import logging


_pinate_datetime_format = '%Y-%m-%dT%H:%M:%S.%fZ'


class PinataUploadResponse(BaseModel):
    IpfsHash: str
    PinSize: int
    Timestamp: datetime.datetime

    @validator('Timestamp', pre=True)
    def parse_foobar(cls, value):
        if isinstance(value, str):
            return datetime.datetime.strptime(value, _pinate_datetime_format)
        return value


class PinataApi:
    """ According to
    https://pinata.cloud/documentation#PinFileToIPFS
    """
    # endpoints
    PINFILE = 'https://api.pinata.cloud/pinning/pinFileToIPFS'
    PINJSON = "https://api.pinata.cloud/pinning/pinJsonToIPFS"

    def __init__(
            self,
            api_key: str = settings.PINATA_CONFIG['api_key'],
            secret_api_key: str = settings.PINATA_CONFIG['secret_api_key'],
    ):
        self.auth_headers = {
            "pinata_api_key": api_key,
            "pinata_secret_api_key": secret_api_key,
        }

    def pin_file2ipfs(self, path: Optional[str] = None, file: bytes = None) -> PinataUploadResponse:
        file = open(path, 'rb') if path else file
        r = requests.post(
            self.PINFILE,
            files={
                'file': file,
            },
            headers=self.auth_headers,
        )
        print(r.json())
        return PinataUploadResponse(**r.json())

    def pin_json2ipfs(self, data: dict) -> PinataUploadResponse:
        r = requests.post(
            self.PINJSON,
            json=data,
            headers=self.auth_headers,
        )
        print(r.json())
        return PinataUploadResponse(**r.json())

In [24]:
from typing import Optional

import logging
import requests
import os


class DiplomaUploader:
    """ According to
    https://docs.rarible.com/asset/creating-an-asset
    """
    pinata = PinataApi()

    def __init__(self):  # todo
        pass
    
    @staticmethod
    def _format_attributes(attributes: dict) -> list:
        attrs = []
        for k,v in attributes.items():
            item = {
                "trait_type": k,
                "value": v
            }
            attrs.append(item)
        return attrs

    def upload_img_get_link(
            self,
            title: str,
            description: str,
            attributes: dict,
            img_path: Optional[str] = None,
            img: Optional[bytes] = None
    ) -> str:
        """ Return hash from uploaded image and meta accroding to
        https://docs.rarible.com/asset/creating-an-asset#uploading-the-image-to-ipfs
        param: user_address: e.g. 0x54210fc276c7b401243a6b921ee30590ed843ac6
        """
        pinned_file = self.pinata.pin_file2ipfs(path=img_path, file=img)

        attributes_packed = DiplomaUploader._format_attributes(attributes)

        data = {
            "attributes" : attributes_packed,
            "description" : description,
            "image" : "https://gateway.pinata.cloud/ipfs/{}".format(pinned_file.IpfsHash),
            "name" : title
        }
        
        metadata_ipfs_hash = self.pinata.pin_json2ipfs(data=data).IpfsHash
        
        return "https://gateway.pinata.cloud/ipfs/{}".format(metadata_ipfs_hash)

In [42]:
import pandas as pd
import numpy as np
import json

In [18]:
df = pd.read_csv("user_attributes.csv")

In [54]:
# df.head()

In [25]:
diploma_uploader = DiplomaUploader()

In [37]:
def add_name(name):
    if not isinstance(name, str) and np.isnan(name):
        return ""
    return name + " as "

In [51]:
links = []
for i in range(df.shape[0]):
    link = diploma_uploader.upload_img_get_link(
            "Itland 4 IT Tracks Course Diploma",
            "This document certifies {}it first owner (upon a creator) has completed the 4 IT Tracks Course by ITLAND school, Jul. 2022".format(add_name(df["names"].values[i])),
            json.loads(df["user_attributes"].values[i].replace("'", '"')),
            "wolfs_gen/{}.png".format(i)
    )
    links.append(link)
    print(i,link)

{'IpfsHash': 'QmSa5fJpSMeVhLr5qkh6XWqJiNu5T3F1XL7XYd9ErYQ5KN', 'PinSize': 306450, 'Timestamp': '2022-07-28T23:13:05.274Z', 'isDuplicate': True}
{'IpfsHash': 'QmUkbhpAYHfHxNPQyBaUB219Gs3zWSSf3mscfLkU7Mdxgn', 'PinSize': 654, 'Timestamp': '2022-07-28T23:15:22.211Z', 'isDuplicate': True}
0 https://gateway.pinata.cloud/ipfs/QmUkbhpAYHfHxNPQyBaUB219Gs3zWSSf3mscfLkU7Mdxgn
{'IpfsHash': 'QmaHCukMepE35bFK6MfNaNepzcNQyhWGjcSPQUykUj8Ub9', 'PinSize': 318822, 'Timestamp': '2022-07-28T23:18:29.473Z'}
{'IpfsHash': 'Qmf4spfkhir9hRK4qMMwUqtd2omm8n9u7Fkqi5KDxxhRHJ', 'PinSize': 676, 'Timestamp': '2022-07-28T23:18:31.085Z'}
1 https://gateway.pinata.cloud/ipfs/Qmf4spfkhir9hRK4qMMwUqtd2omm8n9u7Fkqi5KDxxhRHJ
{'IpfsHash': 'QmeEsgtr1beLXCnKAeQ4YYRxMpzauasxGVXjwAMNPnE9c3', 'PinSize': 313915, 'Timestamp': '2022-07-28T23:18:33.470Z'}
{'IpfsHash': 'QmUEaXwQhCdC7Mer31N1CtGZZHTRcjyp3iY3uGAvFXN7zA', 'PinSize': 658, 'Timestamp': '2022-07-28T23:18:35.117Z'}
2 https://gateway.pinata.cloud/ipfs/QmUEaXwQhCdC7Mer31N1CtGZZHT

In [52]:
links[1]

'https://gateway.pinata.cloud/ipfs/Qmf4spfkhir9hRK4qMMwUqtd2omm8n9u7Fkqi5KDxxhRHJ'

In [53]:
print('\n'.join(links))

https://gateway.pinata.cloud/ipfs/QmUkbhpAYHfHxNPQyBaUB219Gs3zWSSf3mscfLkU7Mdxgn
https://gateway.pinata.cloud/ipfs/Qmf4spfkhir9hRK4qMMwUqtd2omm8n9u7Fkqi5KDxxhRHJ
https://gateway.pinata.cloud/ipfs/QmUEaXwQhCdC7Mer31N1CtGZZHTRcjyp3iY3uGAvFXN7zA
https://gateway.pinata.cloud/ipfs/QmaNBJB1xw12ihsx9fph6Ajqggi4uiD3n6B1g897mHpcFv
https://gateway.pinata.cloud/ipfs/QmQjK5BmBnfGAGYKGQmGizPgzSosMeDTkWhgg6rGT7tJfJ
https://gateway.pinata.cloud/ipfs/QmYvdPCVusq6CqLWyF117Ntp34mQJvianCZarsqgnFCUQn
https://gateway.pinata.cloud/ipfs/QmfZvrGYwQk6UuX9CQ1yzr6eNBPUHmN5og2hiLbckGU5ZZ
https://gateway.pinata.cloud/ipfs/QmUEaXwQhCdC7Mer31N1CtGZZHTRcjyp3iY3uGAvFXN7zA
https://gateway.pinata.cloud/ipfs/QmUEaXwQhCdC7Mer31N1CtGZZHTRcjyp3iY3uGAvFXN7zA
https://gateway.pinata.cloud/ipfs/QmakX31eCHPqoXRF2VUSdHU8Gbf6uzAwRVWBTfoc5LHF6z
https://gateway.pinata.cloud/ipfs/QmXtcwrjifjaKvCv3CmBJRXjwtgBCPYQnebvcAQnNUtG3L
https://gateway.pinata.cloud/ipfs/QmSd7i1hcLiFdtwXMoheCbhkmS6QMLvqpxxmrvn9cxpPPL
https://gateway.pinata.cloud